# Dataset other_train.csv
#### Anton Rusňák, František Gič

In [249]:
import pandas as pd
import numpy as np
import matplotlib as mat
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats as sm_stats
import statsmodels.stats.api as sms
import scipy.stats as stats
%matplotlib inline

data = pd.read_csv("data/other_train.csv",sep=",")

In [252]:
data.head()

,Unnamed: 0,name,address,kurtosis_oxygen,pregnant,education-num,relationship,skewness_glucose,mean_glucose,std_oxygen,...,kurtosis_glucose,personal_info,education,fnlwgt,class,std_glucose,income,mean_oxygen,hours-per-week,capital-loss
0,0,Alex Chinzi,"590 Samantha Bridge Apt. 572\r\nNorth Allison,...",8.286599,f,16.0,Husband,0.234418,107.484375,15.072284,...,0.362969,Prof-specialty|United-States\r\r\nMarried-civ-...,Doctorate,320102.0,0.0,44.487706,>50K,2.448161,50.0,0.0
1,1,Scott Bass,"5384 Hurst Groves Apt. 092\r\nFreemanview, SD ...",6.455860,f,13.0,Own-child,0.151130,140.203125,21.925411,...,-0.170101,Prof-specialty|United-States\r\r\nNever-marrie...,Bachelors,148300.0,0.0,44.586531,<=50K,4.445652,15.0,0.0
2,2,Chuck Lao,USS Scott\r\nFPO AE 78885,11.513491,f,9.0,Unmarried,-0.024384,124.054688,11.822427,...,-0.026966,Other-service|Mexico\r\r\nSeparated -- Private...,HS-grad,190885.0,0.0,45.904165,<=50K,1.890468,40.0,0.0
3,3,Floyd Squires,"122 Ryan Ranch Suite 621\r\nPort Douglas, NM 0...",7.641796,f,13.0,Not-in-family,-0.564287,115.046875,22.965651,...,0.269421,Prof-specialty|Canada\r\r\nNever-married -- Pr...,Bachelors,191385.0,0.0,58.140302,<=50K,3.639632,40.0,0.0
4,4,Kenneth Cadet,"30930 Ryan Groves\r\nKatelynside, MS 75220",8.826058,f,9.0,Husband,0.495896,93.335938,13.777158,...,0.537549,Sales|United-States\r\r\nMarried-civ-spouse --...,HS-grad,226875.0,0.0,45.605865,<=50K,2.510870,40.0,0.0


In [253]:
data = data.drop(data.columns[0],axis=1)


In [257]:
data.describe()


,kurtosis_oxygen,education-num,skewness_glucose,mean_glucose,std_oxygen,capital-gain,skewness_oxygen,kurtosis_glucose,fnlwgt,class,std_glucose,mean_oxygen,hours-per-week,capital-loss
count,3978.000000,3965.000000,3960.000000,3973.000000,3969.000000,3967.000000,3970.000000,3965.000000,3970.000000,3964.000000,3971.000000,3967.000000,3970.000000,3963.000000
mean,7.414473,10.084489,4.242530,101.248940,31.148558,1042.988908,91.370712,0.950507,192967.197985,0.255298,74.479764,18.470601,40.568766,80.275549
std,4.811907,2.588918,9.706012,33.601519,22.263993,7006.726961,108.475749,1.611069,103285.567216,0.436083,1448.165852,34.808695,12.601133,388.336046
min,-2.597872,1.000000,-1.755332,6.187500,7.370432,0.000000,-1.976976,-1.641515,19214.000000,0.000000,-8305.388133,0.213211,1.000000,0.000000
25%,3.630716,9.000000,-0.120081,87.742188,15.024700,0.000000,12.858504,0.072144,120960.000000,0.000000,39.502222,2.061873,40.000000,0.000000
50%,7.784802,10.000000,0.374657,110.664062,20.547873,0.000000,68.623477,0.310372,182045.000000,0.000000,45.928079,3.223244,40.000000,0.000000
75%,10.278883,12.000000,2.091543,124.132812,43.253607,0.000000,127.236918,0.775684,241631.000000,1.000000,51.048246,12.610786,45.000000,0.000000
max,34.539844,16.000000,68.101622,189.734375,110.642211,99999.000000,1191.000837,8.069522,889965.000000,1.000000,6649.471314,217.371238,99.000000,3770.000000


In [ ]:
def iqr(attribute): 
    return data[attribute].quantile(0.75) - data[attribute].quantile(0.25) 

def outliers(attribute):
    iqrange = iqr(attribute)
    return data[(data[attribute] > (data[attribute].quantile(0.75) + iqrange*1.5)) | (data[attribute] < (data[attribute].quantile(0.25) - 1.5*iqrange))]

In [ ]:
def sanitize_boolean(boolean):
    try:
        if boolean.strip() in ['f','F','FALSE','false','False']:
            return False
        elif boolean.strip() in ['t','T','TRUE','true','True']:
            return True
    except AttributeError:
        return None
    
data.pregnant = data.pregnant.apply(sanitize_boolean)


In [303]:
def parse_personal_info(personal_info):
    try:    
        return np.array(personal_info.replace(' -- ', ',').replace('|', ',').replace('\r\r\n', ',').split(','))
    except AttributeError:
        return None
    
def remove_empty(arr):
        if arr is not None:
            empty = ['?','??']
            for i in range(0,len(arr)):
                if (arr[i] != None):
                    if arr[i] in empty:
                        arr[i] = None
        return arr
    
def fill_value(number,arr):
    if arr is not None:
        return arr[number]

In [304]:
parsed_info = data.personal_info.apply(parse_personal_info).apply(remove_empty)

for index,name in enumerate(['employment','country','relationship_info','employment_info','race']):
    data[name] = parsed_info.apply(lambda x: fill_value(index,x))

In [338]:
if (len(duplicatedRows)):
    print("Dataframe obsahuje %d duplicitnych riadkov"%len(duplicatedRows))
else:
    print("Dataframe neobsahuje ziadne duplicitne riadkv")    


Dataframe neobsahuje ziadne duplicitne riadkv


In [356]:
def iqr(attribute): 
    return attribute.quantile(0.75) - attribute.quantile(0.25) 

def outliers(attribute):
    iqrange = iqr(attribute)
    
    lower = attribute.quantile(0.25) - 1.5 * iqrange
    upper = attribute.quantile(0.75) + 1.5 * iqrange
   
    return attribute[(attribute > upper) | (attribute < lower)]